# Day 03 --- Decision Trees: Theory, Practice, and Comparison

Welcome to Day 03 of the ML Track! In Day 02 we built a **logistic regression baseline** and learned
how to evaluate classifiers with metrics like accuracy, precision, recall, F1, and ROC-AUC.

Today we move to our first **non-linear** model: the **Decision Tree**.

## What is a Decision Tree?

A decision tree is a flowchart-like model that makes predictions by asking a sequence of yes/no
questions about the input features. Think of the childhood game **"20 Questions"** --- you narrow
down possibilities by asking the most informative question at each step.

**Real-world use cases:**

| Domain | Example |
|---|---|
| **Credit scoring** | Is annual income > $50k? Is debt-to-income ratio < 0.3? Approve/deny loan. |
| **Medical diagnosis** | Is white blood cell count elevated? Is the patient over 50? Suspect infection vs. other. |
| **Customer churn** | Has the customer called support > 3 times? Is their contract month-to-month? Likely to churn. |
| **Manufacturing QA** | Is temperature > 200C? Is pressure within tolerance? Flag defective part. |

Decision trees are beloved because **you can read them** --- they produce human-interpretable rules.
This makes them invaluable in regulated industries (finance, healthcare) where you must **explain**
why a model made a particular decision.

## What we will cover today

1. Theory: how trees split data (Gini impurity, entropy, information gain)
2. Theory: bias-variance tradeoff in trees
3. Training and evaluating decision trees on the Breast Cancer Wisconsin dataset
4. Depth tuning and pruning to control overfitting
5. Visualizing and interpreting tree structure
6. Feature importance analysis
7. Head-to-head comparison with logistic regression from Day 02
8. Strengths, weaknesses, and a preview of ensemble methods

---
## Theory: How Decision Trees Work

A decision tree learns by performing **recursive binary splitting**:

1. Start with the entire dataset at the **root node**.
2. For every feature and every possible threshold, evaluate how well that split separates the classes.
3. Choose the (feature, threshold) pair that produces the **purest** child nodes.
4. Split the data into two child nodes (left = condition true, right = condition false).
5. Repeat steps 2--4 on each child node **recursively**.
6. Stop when a **stopping criterion** is met (max depth reached, too few samples, node is already pure).

### ASCII Example: A Simple Decision Tree

Imagine we are classifying tumors as malignant (M) or benign (B) using two features:

```
                    [worst radius <= 16.8?]
                    /                      \
                 YES                        NO
                  |                          |
        [mean texture <= 19.5?]         [Predict: MALIGNANT]
        /                    \              (42 samples)
     YES                     NO
      |                       |
 [Predict: BENIGN]    [Predict: MALIGNANT]
   (310 samples)         (17 samples)
```

At each **internal node**, the tree asks a question about a single feature.
At each **leaf node**, the tree outputs a class prediction (the majority class of samples that ended up there).

### Stopping Criteria

Without any limits, a tree will keep splitting until every leaf is perfectly pure (contains only
one class). This leads to **overfitting**. Common stopping criteria include:

- **`max_depth`**: Maximum number of levels from root to any leaf.
- **`min_samples_split`**: Minimum samples required to split an internal node.
- **`min_samples_leaf`**: Minimum samples required in each leaf.
- **`max_leaf_nodes`**: Maximum total number of leaf nodes.

---
## Theory: Splitting Criteria --- Gini Impurity and Entropy

How does the tree decide which feature and threshold to split on? It uses an **impurity measure**
to quantify how mixed the classes are in a node. The goal is to find splits that **reduce impurity
the most**.

### Gini Impurity

The Gini impurity of a node $t$ with $K$ classes is:

$$\text{Gini}(t) = 1 - \sum_{i=1}^{K} p_i^2$$

where $p_i$ is the proportion of samples belonging to class $i$ in node $t$.

**Worked example:** A node has 40 malignant and 60 benign samples (100 total).

- $p_{\text{malignant}} = 40/100 = 0.4$
- $p_{\text{benign}} = 60/100 = 0.6$
- $\text{Gini} = 1 - (0.4^2 + 0.6^2) = 1 - (0.16 + 0.36) = 1 - 0.52 = 0.48$

A **pure node** (all one class) has Gini = 0. The **maximum impurity** for 2 classes is 0.5 (50/50 split).

### Entropy and Information Gain

Entropy measures disorder using information theory:

$$H(t) = -\sum_{i=1}^{K} p_i \log_2(p_i)$$

**Worked example** (same node: 40 malignant, 60 benign):

- $H = -(0.4 \times \log_2 0.4 + 0.6 \times \log_2 0.6)$
- $H = -(0.4 \times (-1.322) + 0.6 \times (-0.737))$
- $H = -(-0.529 + (-0.442))$
- $H = 0.971$ bits

A pure node has entropy = 0. Maximum entropy for 2 classes is 1.0 bit (50/50 split).

**Information Gain** is the reduction in entropy after a split:

$$\text{IG} = H(\text{parent}) - \left[\frac{n_{\text{left}}}{n} H(\text{left}) + \frac{n_{\text{right}}}{n} H(\text{right})\right]$$

The tree picks the split that **maximizes information gain** (or equivalently, maximizes Gini reduction).

### Gini vs Entropy: Comparison

| Aspect | Gini Impurity | Entropy |
|---|---|---|
| **Formula** | $1 - \sum p_i^2$ | $-\sum p_i \log_2 p_i$ |
| **Range (2 classes)** | 0 to 0.5 | 0 to 1.0 |
| **Computation** | Faster (no logarithm) | Slightly slower |
| **Behavior** | Tends to isolate the most frequent class | Tends to produce more balanced trees |
| **Default in sklearn** | Yes (criterion='gini') | No (criterion='entropy') |
| **When to use** | Good default for most problems | When you want slightly more balanced splits |

In practice, the two criteria produce **very similar trees**. Gini is the default because it is
marginally faster to compute.

---
## Theory: The Bias-Variance Tradeoff in Decision Trees

Decision trees provide a textbook illustration of the **bias-variance tradeoff**:

### Deep Trees (Low Bias, High Variance)
- A deep tree can model very complex decision boundaries.
- It has **low bias** because it can fit almost any pattern in the training data.
- But it has **high variance** --- small changes in the training data produce very different trees.
- It **memorizes** the training set, including noise. This is **overfitting**.

### Shallow Trees (High Bias, Low Variance)
- A shallow tree (e.g., depth = 1, called a "decision stump") uses very few splits.
- It has **high bias** because it cannot capture complex patterns.
- But it has **low variance** --- it is stable across different training samples.
- It **underfits** because it is too simple for the true pattern. This is **underfitting**.

### The Sweet Spot (U-Curve of Test Error)

```
  Error
    |  \
    |   \   Test Error
    |    \__________
    |     Sweet      \___________
    |     Spot!        \          \  <-- Overfitting zone
    |                   \
    |    ________________\_________
    |   /  Training Error
    |  /
    +---------------------------------->
         Model Complexity (depth)
```

- As depth increases, **training error** monotonically decreases (eventually to zero).
- **Test error** first decreases (model learns real patterns) then increases (model learns noise).
- The optimal depth is at the **minimum of the test error curve**.

We will see this U-curve clearly in our experiments below.

---
## Setup and Imports

In [ ]:
# ============================================================
# Standard library and data manipulation
# ============================================================
import numpy as np                  # Numerical operations (arrays, math)
import pandas as pd                 # DataFrames for tabular data manipulation

# ============================================================
# Visualization
# ============================================================
import matplotlib.pyplot as plt     # Core plotting library

# ============================================================
# Scikit-learn: dataset
# ============================================================
from sklearn.datasets import load_breast_cancer  # Classic binary classification dataset

# ============================================================
# Scikit-learn: model selection and preprocessing
# ============================================================
from sklearn.model_selection import train_test_split   # Split data into train/test
from sklearn.model_selection import cross_val_score     # K-fold cross-validation
from sklearn.preprocessing import StandardScaler        # Feature scaling for logistic regression

# ============================================================
# Scikit-learn: models
# ============================================================
from sklearn.tree import DecisionTreeClassifier   # The decision tree model
from sklearn.tree import plot_tree, export_text    # Visualization utilities for trees
from sklearn.linear_model import LogisticRegression  # For comparison with Day 02 baseline

# ============================================================
# Scikit-learn: evaluation metrics
# ============================================================
from sklearn.metrics import (
    accuracy_score,          # Overall correct predictions / total predictions
    precision_score,         # TP / (TP + FP) -- how many positive predictions were correct
    recall_score,            # TP / (TP + FN) -- how many actual positives were found
    f1_score,                # Harmonic mean of precision and recall
    roc_auc_score,           # Area under ROC curve -- ranking quality
    classification_report,   # Full precision/recall/f1 report per class
    confusion_matrix,        # 2x2 matrix of TP, TN, FP, FN
)

# ============================================================
# Display settings
# ============================================================
# Ensure plots appear inline in the notebook
%matplotlib inline

# Use a clean plot style for better readability
plt.style.use('seaborn-v0_8-whitegrid')

# Set pandas display options so DataFrames render nicely
pd.set_option('display.max_columns', 35)
pd.set_option('display.width', 200)

print("All imports successful. Ready to go!")

---
## Loading the Dataset

We use the **Breast Cancer Wisconsin (Diagnostic)** dataset, the same one referenced in Day 02.
It contains 569 samples with 30 numeric features computed from digitized images of fine needle
aspirates (FNA) of breast masses. The target is binary: **malignant (0)** or **benign (1)**.

In [ ]:
# Load the dataset from sklearn -- it comes as a Bunch object
cancer = load_breast_cancer()

# Convert to a pandas DataFrame for easier exploration and manipulation
# cancer.data is a 2D numpy array of shape (569, 30)
# cancer.feature_names gives us human-readable column names
X = pd.DataFrame(cancer.data, columns=cancer.feature_names)

# The target variable: 0 = malignant, 1 = benign
y = pd.Series(cancer.target, name='target')

# Print basic info about the dataset
print(f"Dataset shape: {X.shape[0]} samples, {X.shape[1]} features")
print(f"Target classes: {cancer.target_names}")
print(f"  0 -> {cancer.target_names[0]} (malignant)")
print(f"  1 -> {cancer.target_names[1]} (benign)")
print(f"\nFeature names:\n{list(cancer.feature_names)}")

---
## Data Exploration

Before building any model, we should understand our data. For decision trees specifically,
we care about:
- **Class balance**: Imbalanced classes can bias the tree toward the majority class.
- **Feature ranges**: Unlike logistic regression, decision trees do NOT require feature scaling
  (they split on thresholds, so the absolute scale does not matter).
- **Feature distributions**: Helps us understand what the tree might split on.

In [ ]:
# ---- Class balance ----
# Check how many samples belong to each class
# A roughly balanced dataset means accuracy is a reasonable metric
class_counts = y.value_counts()
print("Class distribution:")
print(f"  Benign (1):    {class_counts[1]} samples ({class_counts[1]/len(y)*100:.1f}%)")
print(f"  Malignant (0): {class_counts[0]} samples ({class_counts[0]/len(y)*100:.1f}%)")
print(f"  Ratio (benign/malignant): {class_counts[1]/class_counts[0]:.2f}")
print()

# ---- Summary statistics ----
# Show min, max, mean, std to understand feature ranges
# Notice how features span very different scales (e.g., mean area ~100-2500 vs mean smoothness ~0.05-0.16)
# This does NOT matter for decision trees, but DOES matter for logistic regression
print("Summary statistics (first 10 features):")
X.iloc[:, :10].describe().round(3)

In [ ]:
# ---- Visualize feature ranges by class ----
# Plot the means of the first 10 features for each class
# This helps us anticipate which features might be strong splitters
fig, axes = plt.subplots(2, 5, figsize=(18, 7))
fig.suptitle('Feature Distributions by Class (first 10 features)', fontsize=14, fontweight='bold')

for idx, ax in enumerate(axes.flat):
    feature = cancer.feature_names[idx]  # Get the feature name
    
    # Separate the feature values by class for side-by-side comparison
    malignant_vals = X.loc[y == 0, feature]  # Class 0 = malignant
    benign_vals = X.loc[y == 1, feature]     # Class 1 = benign
    
    # Plot overlapping histograms so we can see separation (or lack thereof)
    ax.hist(malignant_vals, bins=20, alpha=0.6, color='red', label='Malignant')
    ax.hist(benign_vals, bins=20, alpha=0.6, color='steelblue', label='Benign')
    ax.set_title(feature, fontsize=9)
    ax.tick_params(labelsize=7)

# Add a single legend for the entire figure
axes[0, 0].legend(fontsize=8)
plt.tight_layout()
plt.show()

# Features where the two histograms are well-separated (e.g., mean radius, mean concave points)
# will likely appear near the top of the tree (high information gain)

---
## Train/Test Split

We split the data into training (80%) and test (20%) sets. Key points:
- **`stratify=y`** ensures both sets have the same class proportions as the full dataset.
  Without stratification, random chance could give us an unrepresentative test set.
- **`random_state=42`** makes the split reproducible so results are consistent across runs.

In [ ]:
# Perform the train/test split
# test_size=0.2 means 20% of data reserved for testing (~114 samples)
# stratify=y preserves the class ratio in both train and test sets
# random_state=42 ensures reproducibility
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# Verify the split sizes and class proportions
print(f"Training set: {X_train.shape[0]} samples")
print(f"  Benign:    {(y_train == 1).sum()} ({(y_train == 1).mean()*100:.1f}%)")
print(f"  Malignant: {(y_train == 0).sum()} ({(y_train == 0).mean()*100:.1f}%)")
print()
print(f"Test set:  {X_test.shape[0]} samples")
print(f"  Benign:    {(y_test == 1).sum()} ({(y_test == 1).mean()*100:.1f}%)")
print(f"  Malignant: {(y_test == 0).sum()} ({(y_test == 0).mean()*100:.1f}%)")

---
## Theory: Building a Decision Tree with Scikit-Learn

Scikit-learn's `DecisionTreeClassifier` implements the **CART (Classification and Regression Trees)**
algorithm. Here are the key parameters you should know:

| Parameter | Default | What it controls |
|---|---|---|
| `criterion` | `'gini'` | Impurity measure: `'gini'` or `'entropy'` |
| `max_depth` | `None` | Maximum depth of the tree. `None` = no limit (grow until pure) |
| `min_samples_split` | `2` | Minimum samples needed to split an internal node |
| `min_samples_leaf` | `1` | Minimum samples in each leaf node |
| `max_features` | `None` | Number of features to consider at each split. `None` = all features |
| `max_leaf_nodes` | `None` | Maximum number of leaf nodes. `None` = no limit |
| `ccp_alpha` | `0.0` | Complexity parameter for cost-complexity pruning (higher = more pruning) |

The most important parameters for controlling overfitting are **`max_depth`** and **`ccp_alpha`**.
We will experiment with both below.

---
## Training an Unrestricted Tree (Overfitting Demo)

In [ ]:
# ---- Train a decision tree with NO restrictions ----
# max_depth=None means the tree will grow until every leaf is pure
# or until min_samples_split is reached (default=2)
# This typically leads to a very deep tree that overfits the training data
tree_unrestricted = DecisionTreeClassifier(
    random_state=42  # Only set random_state for reproducibility; everything else is default
)

# Fit the tree on training data
tree_unrestricted.fit(X_train, y_train)

# Evaluate on BOTH training and test sets to detect overfitting
train_acc_unres = accuracy_score(y_train, tree_unrestricted.predict(X_train))
test_acc_unres = accuracy_score(y_test, tree_unrestricted.predict(X_test))

print("=== Unrestricted Decision Tree ===")
print(f"Tree depth:      {tree_unrestricted.get_depth()}")
print(f"Number of leaves: {tree_unrestricted.get_n_leaves()}")
print(f"Training accuracy: {train_acc_unres:.4f}")
print(f"Test accuracy:     {test_acc_unres:.4f}")
print(f"Gap (train - test): {train_acc_unres - test_acc_unres:.4f}")
print()
print("OBSERVATION: Training accuracy is 1.0 (perfect) but test accuracy is lower.")
print("This gap is a classic sign of OVERFITTING --- the tree memorized the training data.")

---
## Theory: Overfitting in Decision Trees

Why do unrestricted decision trees overfit so aggressively?

1. **Each leaf can contain as few as 1 sample.** The tree keeps splitting until every training
   example is perfectly classified. This means the tree creates hyper-specific rules that
   only apply to individual training points.

2. **The tree memorizes noise.** Real data always contains some noise (measurement errors,
   natural variability). An unrestricted tree will create splits to accommodate this noise,
   learning patterns that do not generalize.

3. **High model complexity.** A tree with hundreds of leaves is essentially a lookup table.
   It has too many parameters relative to the amount of training data.

### How depth controls complexity

| Depth | Max Leaves | Complexity | Risk |
|---|---|---|---|
| 1 | 2 | Very low | Underfitting |
| 3 | 8 | Low-medium | Usually good |
| 5 | 32 | Medium | Often good |
| 10 | 1024 | High | Potential overfitting |
| None (unlimited) | Up to N | Very high | Almost certain overfitting |

The number of possible leaves grows **exponentially** with depth ($2^d$), so even small
increases in depth can dramatically increase model complexity.

---
## Depth Tuning Experiment

Let us systematically train trees at depths 1 through 15 and plot the training and test
accuracy curves. We expect to see the classic bias-variance U-curve on the test set.

In [ ]:
# ---- Depth tuning: train trees at depths 1 through 15 ----
# We store both train and test accuracy at each depth
# so we can later plot the overfitting curve

depths = list(range(1, 16))  # Depths from 1 to 15
train_accuracies = []        # Will hold training accuracy for each depth
test_accuracies = []         # Will hold test accuracy for each depth

for d in depths:
    # Create a tree constrained to this specific depth
    tree_d = DecisionTreeClassifier(max_depth=d, random_state=42)
    
    # Train on the training set
    tree_d.fit(X_train, y_train)
    
    # Evaluate on both sets
    train_acc = accuracy_score(y_train, tree_d.predict(X_train))
    test_acc = accuracy_score(y_test, tree_d.predict(X_test))
    
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)

# Display results as a table for easy inspection
results_df = pd.DataFrame({
    'depth': depths,
    'train_accuracy': train_accuracies,
    'test_accuracy': test_accuracies,
    'gap': [tr - te for tr, te in zip(train_accuracies, test_accuracies)]
})

print("Depth Tuning Results:")
print(results_df.to_string(index=False))

# Find the depth with the best test accuracy
best_idx = np.argmax(test_accuracies)
best_depth = depths[best_idx]
best_test_acc = test_accuracies[best_idx]
print(f"\nBest test accuracy: {best_test_acc:.4f} at depth = {best_depth}")

---
## Plotting the Depth vs Accuracy Curves

In [ ]:
# ---- Visualization: Train vs Test accuracy as a function of tree depth ----
# This plot is one of the most important diagnostics in machine learning.
# It shows the bias-variance tradeoff in action.

fig, ax = plt.subplots(figsize=(10, 6))

# Plot training accuracy (should increase monotonically with depth)
ax.plot(depths, train_accuracies, 'o-', color='steelblue', linewidth=2,
        markersize=7, label='Training Accuracy')

# Plot test accuracy (should increase, peak, then potentially decrease)
ax.plot(depths, test_accuracies, 's-', color='darkorange', linewidth=2,
        markersize=7, label='Test Accuracy')

# Mark the optimal depth with a vertical dashed line
ax.axvline(x=best_depth, color='green', linestyle='--', alpha=0.7,
           label=f'Best Depth = {best_depth}')

# Annotate the best test accuracy point
ax.annotate(
    f'Best: {best_test_acc:.3f}\n(depth={best_depth})',
    xy=(best_depth, best_test_acc),
    xytext=(best_depth + 2, best_test_acc - 0.03),
    fontsize=11,
    arrowprops=dict(arrowstyle='->', color='green'),
    color='green',
    fontweight='bold'
)

# Add shaded regions to indicate underfitting and overfitting zones
ax.axvspan(0.5, 2.5, alpha=0.08, color='blue', label='Underfitting zone')
ax.axvspan(8.5, 15.5, alpha=0.08, color='red', label='Overfitting zone')

# Labels, title, legend
ax.set_xlabel('Tree Depth', fontsize=13)
ax.set_ylabel('Accuracy', fontsize=13)
ax.set_title('Decision Tree: Training vs Test Accuracy by Depth', fontsize=14, fontweight='bold')
ax.set_xticks(depths)
ax.legend(fontsize=10, loc='lower right')
ax.set_ylim(0.85, 1.02)

plt.tight_layout()
plt.show()

print("INTERPRETATION:")
print("- Training accuracy rises quickly to 1.0 as depth increases (the tree memorizes training data).")
print("- Test accuracy peaks at an intermediate depth, then plateaus or drops.")
print("- The gap between train and test accuracy widens with depth = classic overfitting signature.")
print(f"- The optimal depth for this dataset appears to be around {best_depth}.")

---
## Theory: Tree Pruning

There are two main approaches to controlling tree complexity:

### Pre-Pruning (Early Stopping)
Stop growing the tree before it becomes too complex. This is what `max_depth`,
`min_samples_split`, and `min_samples_leaf` do.

- **Pros**: Simple, fast, easy to tune.
- **Cons**: May stop too early and miss useful splits deeper in the tree.

### Post-Pruning (Cost-Complexity Pruning)
Grow a full tree first, then **remove branches** that do not improve generalization.
Scikit-learn implements this via the **`ccp_alpha`** parameter.

The idea: for each subtree, compute

$$R_{\alpha}(T) = R(T) + \alpha \cdot |T|$$

where:
- $R(T)$ = total impurity of all leaves (training error)
- $|T|$ = number of leaves (complexity)
- $\alpha$ = the complexity parameter (`ccp_alpha`)

A higher $\alpha$ penalizes complexity more heavily, resulting in a smaller tree.
When $\alpha = 0$, we get the full tree. As $\alpha$ increases, branches are pruned.

We can find the optimal $\alpha$ using **cross-validation**.

---
## Cost-Complexity Pruning with Cross-Validation

In [ ]:
# ---- Cost-Complexity Pruning ----
# Step 1: Get the effective alpha values for the full tree
# cost_complexity_pruning_path() returns the sequence of alpha values
# and corresponding impurities as we prune the tree progressively
tree_full = DecisionTreeClassifier(random_state=42)
tree_full.fit(X_train, y_train)

# This gives us the pruning path: a sequence of (alpha, impurity) pairs
path = tree_full.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas  # Array of alpha values from 0 (full tree) to max
impurities = path.impurities  # Corresponding total leaf impurities

print(f"Number of alpha values to evaluate: {len(ccp_alphas)}")
print(f"Alpha range: {ccp_alphas.min():.6f} to {ccp_alphas.max():.6f}")

# Step 2: For each alpha, train a tree and evaluate with 5-fold cross-validation
# We use cross-validation instead of a single train/test split for more robust estimates
cv_scores_mean = []
cv_scores_std = []

for alpha in ccp_alphas:
    tree_cv = DecisionTreeClassifier(ccp_alpha=alpha, random_state=42)
    # cross_val_score performs 5-fold CV and returns accuracy for each fold
    scores = cross_val_score(tree_cv, X_train, y_train, cv=5, scoring='accuracy')
    cv_scores_mean.append(scores.mean())
    cv_scores_std.append(scores.std())

# Convert to numpy arrays for easier manipulation
cv_scores_mean = np.array(cv_scores_mean)
cv_scores_std = np.array(cv_scores_std)

# Find the best alpha (highest mean CV accuracy)
best_alpha_idx = np.argmax(cv_scores_mean)
best_alpha = ccp_alphas[best_alpha_idx]
print(f"\nBest ccp_alpha: {best_alpha:.6f}")
print(f"Best CV accuracy: {cv_scores_mean[best_alpha_idx]:.4f} (+/- {cv_scores_std[best_alpha_idx]:.4f})")

In [ ]:
# ---- Plot CV accuracy vs ccp_alpha ----
fig, ax = plt.subplots(figsize=(10, 6))

# Plot mean CV accuracy with error band (1 standard deviation)
ax.plot(ccp_alphas, cv_scores_mean, 'o-', color='steelblue', markersize=3, label='Mean CV Accuracy')
ax.fill_between(ccp_alphas,
                cv_scores_mean - cv_scores_std,
                cv_scores_mean + cv_scores_std,
                alpha=0.2, color='steelblue', label='+/- 1 Std Dev')

# Mark the best alpha
ax.axvline(x=best_alpha, color='green', linestyle='--', alpha=0.7, label=f'Best alpha={best_alpha:.4f}')

ax.set_xlabel('ccp_alpha (Complexity Parameter)', fontsize=12)
ax.set_ylabel('Cross-Validation Accuracy', fontsize=12)
ax.set_title('Cost-Complexity Pruning: CV Accuracy vs Alpha', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)

plt.tight_layout()
plt.show()

print("INTERPRETATION:")
print("- At alpha=0 (full tree), the tree is complex and may overfit.")
print("- As alpha increases, the tree is pruned and CV accuracy first improves, then drops.")
print(f"- The optimal alpha ({best_alpha:.4f}) balances complexity and performance.")

---
## Training the Optimal Tree

We now train the final decision tree using the best depth we found, and also evaluate the
pruned tree. We will pick whichever performs better on the test set.

In [ ]:
# ---- Train the depth-tuned tree ----
tree_depth_tuned = DecisionTreeClassifier(max_depth=best_depth, random_state=42)
tree_depth_tuned.fit(X_train, y_train)

# ---- Train the pruned tree ----
tree_pruned = DecisionTreeClassifier(ccp_alpha=best_alpha, random_state=42)
tree_pruned.fit(X_train, y_train)

# ---- Evaluate all three trees on the test set ----
models = {
    'Unrestricted': tree_unrestricted,
    f'Depth-Tuned (depth={best_depth})': tree_depth_tuned,
    f'Pruned (alpha={best_alpha:.4f})': tree_pruned,
}

print(f"{'Model':<35} {'Train Acc':>10} {'Test Acc':>10} {'Depth':>7} {'Leaves':>8}")
print('-' * 72)

best_model_name = None
best_model_test_acc = 0
best_model_obj = None

for name, model in models.items():
    tr_acc = accuracy_score(y_train, model.predict(X_train))
    te_acc = accuracy_score(y_test, model.predict(X_test))
    depth = model.get_depth()
    leaves = model.get_n_leaves()
    print(f"{name:<35} {tr_acc:>10.4f} {te_acc:>10.4f} {depth:>7} {leaves:>8}")
    
    # Track the best model by test accuracy
    if te_acc > best_model_test_acc:
        best_model_test_acc = te_acc
        best_model_name = name
        best_model_obj = model

print(f"\nBest decision tree model: {best_model_name} (test accuracy: {best_model_test_acc:.4f})")

---
## Theory: Visualizing Decision Trees

One of the biggest advantages of decision trees is **interpretability**. We can literally
read the decision rules the model learned.

When you visualize a tree, each node shows:

| Element | Meaning |
|---|---|
| **Feature <= threshold** | The splitting condition (samples going left satisfy this) |
| **gini** or **entropy** | The impurity of this node |
| **samples** | Number of training samples that reached this node |
| **value = [a, b]** | Number of samples in each class [malignant, benign] |
| **class** | The majority class prediction at this node |

**Reading a tree:**
- Start at the root (top).
- If the condition is True, go **left**. If False, go **right**.
- Repeat until you reach a leaf node --- that is the prediction.
- The color intensity indicates confidence: darker = more pure.

---
## Tree Visualization

In [ ]:
# ---- Visual tree plot ----
# We visualize a shallow tree (depth=3) for readability
# A full tree would be too wide to display clearly
tree_visual = DecisionTreeClassifier(max_depth=3, random_state=42)
tree_visual.fit(X_train, y_train)

fig, ax = plt.subplots(figsize=(20, 10))

# plot_tree draws the tree structure with all node information
# filled=True colors nodes by majority class (helps see the split pattern)
# rounded=True makes the boxes look nicer
# fontsize controls readability
plot_tree(
    tree_visual,
    feature_names=cancer.feature_names,   # Use real feature names instead of indices
    class_names=cancer.target_names,       # Use 'malignant'/'benign' instead of 0/1
    filled=True,                           # Color nodes by class
    rounded=True,                          # Round box corners
    fontsize=9,                            # Font size for node text
    ax=ax                                  # Draw on our axes
)

ax.set_title('Decision Tree Visualization (max_depth=3)', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("HOW TO READ THIS TREE:")
print("- Start at the root node (top center).")
print("- If the condition is TRUE, follow the LEFT branch.")
print("- If the condition is FALSE, follow the RIGHT branch.")
print("- Orange nodes lean toward malignant; blue nodes lean toward benign.")
print("- Darker color = higher purity (more confident prediction).")

In [ ]:
# ---- Text representation of the tree ----
# export_text produces a text-based representation that is useful for
# documentation, logging, or when you cannot render graphics
tree_text = export_text(
    tree_visual,
    feature_names=list(cancer.feature_names)  # Must be a list, not numpy array
)

print("Text representation of the decision tree (depth=3):")
print("=" * 60)
print(tree_text)
print("=" * 60)
print()
print("Each indentation level represents one level deeper in the tree.")
print("'|---' lines show the path for when the condition is TRUE (left child).")
print("'|---' lines after the condition show the path for FALSE (right child).")
print("'class:' at the leaf nodes shows the predicted class.")

---
## Theory: Feature Importance in Decision Trees

Decision trees provide a natural measure of **feature importance** based on how much each
feature contributes to reducing impurity across all splits in the tree.

### How it is calculated

For each feature, the importance is computed as:

$$\text{Importance}(f) = \sum_{\text{nodes splitting on } f} \frac{n_t}{n} \cdot \Delta \text{impurity}$$

where:
- $n_t$ = number of samples reaching node $t$
- $n$ = total number of training samples
- $\Delta \text{impurity}$ = reduction in impurity (Gini or entropy) caused by this split

The importances are then **normalized to sum to 1.0**.

### Limitations to be aware of

1. **Bias toward high-cardinality features**: Features with many unique values get more
   possible split points, so they may appear more important than they truly are.
2. **Correlated features**: If two features are highly correlated, the tree may use one
   and completely ignore the other, even though both are informative.
3. **Instability**: Small changes in the data can produce very different trees, which
   changes the importance rankings. Ensemble methods (Random Forest) are more stable.

---
## Feature Importance Analysis

In [ ]:
# ---- Extract and plot feature importances from the best tree ----
# We use the best model from our comparison above
importances = best_model_obj.feature_importances_  # Array of shape (30,), sums to 1.0

# Create a DataFrame for easy sorting and display
importance_df = pd.DataFrame({
    'feature': cancer.feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)  # Sort by importance, highest first

# Show only features with non-zero importance
# (many features may have importance=0 if the tree never split on them)
nonzero = importance_df[importance_df['importance'] > 0]
print(f"Features used by the tree: {len(nonzero)} out of {len(cancer.feature_names)}")
print(f"Features NOT used: {len(cancer.feature_names) - len(nonzero)}")
print()
print("Top 10 most important features:")
print(nonzero.head(10).to_string(index=False))

In [ ]:
# ---- Bar plot of feature importances ----
# Show the top 15 features for clarity (plotting all 30 can be cluttered)
top_n = 15
top_features = importance_df.head(top_n)

fig, ax = plt.subplots(figsize=(10, 7))

# Horizontal bar chart: longest bars at the top
bars = ax.barh(
    range(len(top_features)),
    top_features['importance'].values,
    color='steelblue',
    edgecolor='white'
)

# Label each bar with the feature name
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features['feature'].values, fontsize=10)
ax.invert_yaxis()  # Highest importance at the top

# Add value labels on the bars for precise reading
for bar_idx, bar in enumerate(bars):
    width = bar.get_width()
    if width > 0.01:  # Only label bars that are visible
        ax.text(width + 0.005, bar.get_y() + bar.get_height()/2,
                f'{width:.3f}', va='center', fontsize=9)

ax.set_xlabel('Feature Importance (Gini reduction)', fontsize=12)
ax.set_title(f'Top {top_n} Feature Importances ({best_model_name})',
             fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("INTERPRETATION:")
print(f"- The most important feature is '{top_features.iloc[0]['feature']}' "
      f"(importance={top_features.iloc[0]['importance']:.3f}).")
print("- 'Worst' features (computed from the largest/most extreme cell nuclei) tend to")
print("  dominate because they capture the most extreme differences between malignant and benign.")
print("- Many features have zero importance -- the tree found them unnecessary for classification.")

---
## Theory: Decision Trees vs Logistic Regression

Now let us compare the two models we have studied so far.

| Aspect | Decision Tree | Logistic Regression |
|---|---|---|
| **Decision boundary** | Non-linear (axis-aligned rectangles) | Linear (hyperplane) |
| **Interpretability** | Very high (readable rules) | Moderate (coefficients + odds ratios) |
| **Feature scaling needed?** | No (splits on thresholds) | Yes (gradient-based optimization) |
| **Handles non-linear relationships?** | Yes, naturally | No, needs feature engineering |
| **Handles feature interactions?** | Yes (nested splits) | No (unless explicitly added) |
| **Robustness to outliers** | High (threshold-based splits) | Moderate |
| **Stability** | Low (small data changes -> different tree) | High (stable coefficients) |
| **Overfitting risk** | High (if unrestricted) | Lower (regularization built in) |
| **Probabilistic output** | Crude (leaf proportions) | Smooth (sigmoid function) |
| **Training speed** | Fast | Fast |

Neither model is universally better. The right choice depends on your data and requirements.

---
## Comparison with Logistic Regression

In [ ]:
# ---- Train a logistic regression model for head-to-head comparison ----
# Logistic regression requires feature scaling because it uses gradient descent
# and features are on very different scales (e.g., area ~100-2500 vs smoothness ~0.05-0.16)

# Step 1: Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Fit on training data, then transform
X_test_scaled = scaler.transform(X_test)          # Only transform test data (no fitting!)

# Step 2: Train logistic regression
# max_iter=10000 ensures convergence (default 100 may not be enough for 30 features)
logreg = LogisticRegression(max_iter=10000, random_state=42)
logreg.fit(X_train_scaled, y_train)

# Step 3: Get predictions from both models
# Decision tree predictions (using our best tree)
tree_preds = best_model_obj.predict(X_test)
tree_probs = best_model_obj.predict_proba(X_test)[:, 1]  # Probability of benign class

# Logistic regression predictions
lr_preds = logreg.predict(X_test_scaled)
lr_probs = logreg.predict_proba(X_test_scaled)[:, 1]  # Probability of benign class

print("Models trained successfully. Ready for comparison.")

In [ ]:
# ---- Side-by-side metric comparison ----
# We compute the same metrics for both models and display them in a table

def compute_metrics(y_true, y_pred, y_prob):
    """Compute a dictionary of classification metrics."""
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1 Score': f1_score(y_true, y_pred),
        'ROC-AUC': roc_auc_score(y_true, y_prob),
    }

# Compute metrics for both models
tree_metrics = compute_metrics(y_test, tree_preds, tree_probs)
lr_metrics = compute_metrics(y_test, lr_preds, lr_probs)

# Create a comparison DataFrame
comparison_df = pd.DataFrame({
    'Metric': list(tree_metrics.keys()),
    f'Decision Tree ({best_model_name})': [f"{v:.4f}" for v in tree_metrics.values()],
    'Logistic Regression': [f"{v:.4f}" for v in lr_metrics.values()],
})

print("=" * 70)
print("HEAD-TO-HEAD COMPARISON: Decision Tree vs Logistic Regression")
print("=" * 70)
print(comparison_df.to_string(index=False))
print()

# Determine winner for each metric
for metric in tree_metrics:
    tree_val = tree_metrics[metric]
    lr_val = lr_metrics[metric]
    if abs(tree_val - lr_val) < 0.001:
        print(f"  {metric}: TIE")
    elif tree_val > lr_val:
        print(f"  {metric}: Decision Tree wins (+{tree_val - lr_val:.4f})")
    else:
        print(f"  {metric}: Logistic Regression wins (+{lr_val - tree_val:.4f})")

In [ ]:
# ---- Confusion matrices side by side ----
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, preds, title in zip(
    axes,
    [tree_preds, lr_preds],
    [f'Decision Tree ({best_model_name})', 'Logistic Regression']
):
    # Compute the confusion matrix
    cm = confusion_matrix(y_test, preds)
    
    # Display as a heatmap using matplotlib
    im = ax.imshow(cm, cmap='Blues', interpolation='nearest')
    
    # Add text annotations showing the counts in each cell
    for i in range(2):
        for j in range(2):
            # Choose text color based on background intensity
            color = 'white' if cm[i, j] > cm.max() / 2 else 'black'
            ax.text(j, i, str(cm[i, j]), ha='center', va='center',
                    fontsize=20, fontweight='bold', color=color)
    
    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(['Malignant', 'Benign'], fontsize=11)
    ax.set_yticklabels(['Malignant', 'Benign'], fontsize=11)
    ax.set_xlabel('Predicted', fontsize=12)
    ax.set_ylabel('Actual', fontsize=12)
    ax.set_title(title, fontsize=13, fontweight='bold')

plt.suptitle('Confusion Matrix Comparison', fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("READING THE CONFUSION MATRIX:")
print("- Top-left (TN): Correctly identified malignant tumors")
print("- Top-right (FP): Malignant tumors incorrectly predicted as benign (DANGEROUS in medicine!)")
print("- Bottom-left (FN): Benign tumors incorrectly predicted as malignant")
print("- Bottom-right (TP): Correctly identified benign tumors")

---
## Theory: Strengths and Weaknesses of Decision Trees

### Strengths

1. **Highly interpretable**: You can read the decision rules. Crucial for regulated industries.
2. **No feature scaling required**: Trees split on thresholds, so the absolute scale is irrelevant.
3. **Handles non-linear relationships**: Can capture complex patterns that linear models miss.
4. **Handles feature interactions**: Nested splits naturally model interactions.
5. **Works with both numerical and categorical features** (sklearn requires encoding, but the
   algorithm itself handles both).
6. **Fast training and prediction**: Even on large datasets.
7. **Built-in feature selection**: Only informative features are used for splits.

### Weaknesses

1. **Prone to overfitting**: Without pruning, trees memorize training data.
2. **High variance / instability**: Small changes in data can produce a completely different tree.
3. **Axis-aligned splits only**: Decision boundaries are always perpendicular to feature axes.
   Diagonal boundaries require many splits to approximate.
4. **Greedy algorithm**: Each split is locally optimal but may not be globally optimal.
5. **Biased toward features with many values**: More split points = more chances to look useful.

### Preview: Ensemble Methods (What Comes Next)

The weaknesses of individual trees are addressed by **ensemble methods** that combine
many trees:

| Method | How it works | Fixes |
|---|---|---|
| **Random Forest** | Trains many trees on random subsets of data and features, averages predictions | Reduces variance, improves stability |
| **Gradient Boosting** (XGBoost, LightGBM) | Trains trees sequentially, each correcting the previous one's errors | Reduces bias and variance |
| **AdaBoost** | Trains trees on weighted data, upweighting misclassified samples | Reduces bias |

These ensemble methods are among the **most powerful** algorithms in machine learning
and dominate tabular data competitions (e.g., Kaggle).

---
## Summary of Findings

Here is what we learned in this notebook:

### Theory
- Decision trees learn by **recursive binary splitting**, choosing the (feature, threshold) pair
  that maximizes purity at each step.
- **Gini impurity** and **entropy** are the two main splitting criteria. They produce similar
  results; Gini is marginally faster.
- Unrestricted trees **overfit** by memorizing training data. The bias-variance tradeoff tells us
  there is an optimal complexity that minimizes test error.
- **Pre-pruning** (max_depth, min_samples) and **post-pruning** (ccp_alpha) are the two main
  strategies for controlling tree complexity.

### Practice
- An unrestricted tree achieved **100% training accuracy** but lower test accuracy (overfitting).
- Depth tuning revealed a sweet spot, and we visualized the classic train/test accuracy curves.
- Cost-complexity pruning with cross-validation provided another way to find the optimal tree.
- Feature importance analysis revealed which features the tree relies on most heavily.

### Model Comparison
- We compared our best decision tree against logistic regression on the same test set.
- Both models perform well on this dataset. Logistic regression may have an edge in ROC-AUC
  (smoother probability estimates), while the decision tree offers superior interpretability.
- The decision tree requires **no feature scaling**, which simplifies preprocessing.

### Key Takeaway
> A decision tree is a powerful, interpretable model, but it must be carefully constrained
> (via depth limits or pruning) to generalize well. Its weaknesses (instability, overfitting)
> are addressed by ensemble methods, which we will encounter later in the ML Track.

---
## Next Steps: Connection to Day 04 (Feature Engineering)

In **Day 04**, we will explore **Feature Engineering** --- the art of transforming raw features
into more informative representations that help models learn better.

Feature engineering is often the single biggest lever for improving model performance.
It matters for both decision trees and logistic regression:

- **For logistic regression**: Adding polynomial features or interaction terms can help it
  capture non-linear relationships (which decision trees handle naturally).
- **For decision trees**: Creating ratio features (e.g., `area / perimeter`) can help the tree
  find splits that would otherwise require many levels of nesting.

We will continue using the Breast Cancer dataset so we can directly compare how feature
engineering affects the models we have already built.

**See you in Day 04!**